# 이미지 처리 추가

## 추가 방법

### 라이브러리 로드

In [1]:
import cv2
import numpy as np

### 고양이 부르기

In [3]:
cat1 = cv2.imread('./data/cat01.jpg', cv2.IMREAD_COLOR)
cat2 = cv2.imread('./data/cat02.jpg', cv2.IMREAD_COLOR)

### 투명도 합성

In [4]:
# 두 이미지를 50:50 합성
blended = cv2.addWeighted(cat1, 0.5, cat2, 0.5, 0)

cv2.imshow('Transparency Blending', blended)
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src='../images/result047.jpg' width='650'>

### 이미지 블렌딩 + 워터마크

In [16]:
# 원본 이미지
cat = cv2.imread('./data/cat02.jpg', cv2.IMREAD_COLOR)

In [10]:
# 워터마크로 쓸 반투명 사각형
overlay = cat.copy()
cv2.rectangle(overlay, (50, 200), (450, 280), (0, 0, 0), -1)

# addWeighted로 투명 워터마크 합성
watermarked = cv2.addWeighted(overlay, 0.3, cat, 0.7, 0)

# 텍스트 워터마크
cv2.putText(watermarked, 'Sleeping Kitty', (60, 260),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

array([[[  9,  11,  21],
        [  3,   6,  14],
        [  0,   5,  10],
        ...,
        [ 59,  94, 120],
        [ 95, 132, 160],
        [ 69, 107, 137]],

       [[ 15,  17,  28],
        [  5,   7,  17],
        [  0,   3,  10],
        ...,
        [ 64,  99, 125],
        [ 82, 119, 147],
        [ 72, 110, 140]],

       [[ 15,  17,  35],
        [  2,   5,  20],
        [  0,   1,  10],
        ...,
        [ 35,  72,  98],
        [ 94, 133, 161],
        [ 67, 107, 136]],

       ...,

       [[ 78, 110, 133],
        [ 42,  73,  96],
        [ 51,  79, 103],
        ...,
        [105, 158, 195],
        [111, 164, 201],
        [ 85, 140, 177]],

       [[105, 137, 160],
        [ 81, 113, 136],
        [ 83, 114, 137],
        ...,
        [ 53, 106, 143],
        [110, 165, 202],
        [ 91, 146, 183]],

       [[ 68, 101, 121],
        [ 48,  81, 101],
        [ 41,  73,  96],
        ...,
        [ 80, 134, 169],
        [ 93, 149, 184],
        [102, 160, 195]]

In [11]:
cv2.imshow('Watermarked Image', watermarked)
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src='../images/result048.jpg' width='650'>

## 이진화

In [25]:
# 워터마크로 쓸 반투명 사각형
copy = cat.copy()

# 2. 그레이스케일 변환
gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)

# 이진화 적용
_, th_binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
_, th_inv    = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

# 단일 채널 이미지를 컬러(BGR)로 변환해야 합칠 수 있음
gray_color     = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
binary_color   = cv2.cvtColor(th_binary, cv2.COLOR_GRAY2BGR)
inv_color      = cv2.cvtColor(th_inv, cv2.COLOR_GRAY2BGR)

# 세로 방향으로 합치기
stacked = cv2.vconcat([binary_color, inv_color])

cv2.imshow('original', gray_color)
cv2.imshow('Result', stacked)
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src='../images/result049.jpg' width='650'>

<img src='../images/result050.jpg' width='650'>

## 마스크

In [26]:
# 1) 이미지 불러오기
img = cv2.imread('./data/cat02.jpg', cv2.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError('이미지를 불러올 수 없습니다.')

h, w = img.shape[:2]

# 2) 마스크 이미지 생성 (검은색 배경)
mask = np.zeros((h, w), dtype=np.uint8)

# 3) 텍스트로 "MASK" 작성
cv2.putText(mask, 'MASK', (w//4, h//2),
            cv2.FONT_HERSHEY_SIMPLEX, 5, 255, 20, cv2.LINE_AA)

# 4) 마스크 적용 (흰 부분만 남김)
masked_fg = cv2.bitwise_and(img, img, mask=mask)

# 5) 배경(회색) 준비
background = np.full_like(img, (180, 180, 180))
mask_inv = cv2.bitwise_not(mask)
bg_part = cv2.bitwise_and(background, background, mask=mask_inv)

# 6) 최종 합성
result = cv2.add(masked_fg, bg_part)

# 7) 비교 출력
row1 = cv2.hconcat([img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)])
row2 = cv2.hconcat([masked_fg, result])
montage = cv2.vconcat([row1, row2])

cv2.imshow('Original | Mask / Foreground | Result', montage)
cv2.waitKey(0)
cv2.destroyAllWindows()

# (선택) 저장
cv2.imwrite('./result/mask_text_demo.jpg', montage)

True

<img src="../images/result051.jpg" width="700">